In [ ]:
%matplotlib notebook
import numpy as np
import scipy.signal
import matplotlib.pylab as plt
from matplotlib import animation, patches
import ipywidgets as widgets
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80
from IPython.display import YouTubeVideo

## Apéndice: Procesamiento de señales de audio con Scipy y soundfile

Referencias
- https://docs.scipy.org/doc/scipy/reference/signal.html
- https://pysoundfile.readthedocs.io/en/0.9.0/

In [ ]:
# Espectrograma de una sinusoide pura y de una sinusoide modulada (tremolo y vibrato)
plt.close('all'); fig, ax = plt.subplots(2, figsize=(8, 6))
sample_rate = 44100; time = np.arange(0, 3, step=1.0/sample_rate)
noise = 0.0*np.random.randn(len(time))
mod_am = 1.0 # 0.25 + (1.0 + np.cos(2*np.pi*4*time))*0.75
mod_fm = 20*np.cos(2.*np.pi*8*time);
phase_fix = np.add.accumulate(time*np.concatenate((np.zeros(1), 2*np.pi*(mod_fm[:-1]-mod_fm[1:]))))
data = mod_am*np.sin(2.*np.pi*(220 + mod_fm)*time + phase_fix) + noise
ax[0].plot(time, data); 
freq, ttime, Sxx = scipy.signal.spectrogram(data, fs=sample_rate, window=('tukey', 0.25), 
                                            nperseg=256, noverlap=None, detrend=False,
                                            return_onesided=True, scaling='density', mode='magnitude')
ax[1].pcolormesh(ttime, freq, np.log10(Sxx + 1e-5), cmap=plt.cm.magma); #ax[1].set_ylim([0., 5e+3]) 
Audio(data, rate=sample_rate)

In [ ]:
sample_rate

In [ ]:
# Espectrograma de audio sintético
plt.close('all'); fig, ax = plt.subplots(2, figsize=(8, 6))
data, sample_rate = sf.read("data/DPSAU.ogg")
time = np.linspace(0.0, len(data)/sample_rate, num=len(data))
ax[0].plot(time, data);
freq, ttime, Sxx = scipy.signal.spectrogram(data, fs=sample_rate, window=('tukey', 0.25), 
                                            nperseg=2048, noverlap=512, detrend=False,
                                            return_onesided=True, scaling='density', mode='magnitude')
ax[1].pcolormesh(ttime, freq, np.log10(Sxx+1e-3), cmap=plt.cm.magma); ax[1].set_ylim([0.0, 2e+3]);

Audio(data, rate=sample_rate)

In [ ]:
# Espectrograma de una señal de voz
plt.close('all'); fig, ax = plt.subplots(2, figsize=(8, 6))
data, sample_rate = sf.read("data/123.ogg")
time = np.linspace(0.0, len(data)/sample_rate, num=len(data))
ax[0].plot(time, data);
freq, ttime, Sxx = scipy.signal.spectrogram(data, fs=sample_rate, window=('tukey', 0.25), 
                                            nperseg=256, noverlap=None, detrend=False,
                                            return_onesided=True, scaling='density', mode='magnitude')
ax[1].pcolormesh(ttime, freq, np.log10(Sxx+1e-5), cmap=plt.cm.magma); ax[1].set_ylim([0.0, 5e+3]);
Audio(data, rate=sample_rate)

In [ ]:
# Filtro de polos y zeros en frecuencia para una señal de voz
plt.close('all'); fig, ax = plt.subplots(1, figsize=(8, 3))
b, a = scipy.signal.butter(7, 0.01, btype='lowpass')
data_filt = scipy.signal.lfilter(b, a, data)

freq, ttime, Sxx = scipy.signal.spectrogram(data_filt, fs=sample_rate, window=('tukey', 0.25), 
                                            nperseg=256, noverlap=None, detrend=False,
                                            return_onesided=True, scaling='density', mode='magnitude')
ax.pcolormesh(ttime, freq, np.log10(Sxx+1e-5), cmap=plt.cm.magma); ax.set_ylim([0.0, 5e+3]);
Audio(data_filt, rate=sample_rate)

In [ ]:
# Filtro de polos y zeros hecho "a mano" para dar un efecto de eco
plt.close('all'); fig, ax = plt.subplots(1, figsize=(8, 3))
b = np.zeros(shape=(sample_rate//2,)); b[0] = 1
for p in range(0, len(b)+1, len(b)):
    b[p-1] = 0.5

data_filt = scipy.signal.lfilter(b, np.array([1.0, 0.0]), np.hstack((data, np.zeros(shape=(sample_rate,)))))
freq, ttime, Sxx = scipy.signal.spectrogram(data_filt, fs=sample_rate, window=('tukey', 0.25), 
                                            nperseg=256, noverlap=None, detrend=False,
                                            return_onesided=True, scaling='density', mode='magnitude')
ax.pcolormesh(ttime, freq, np.log10(Sxx+1e-5), cmap=plt.cm.magma); ax.set_ylim([0.0, 5e+3]);
Audio(data_filt, rate=sample_rate)

In [ ]:
# Ref: https://ipython-books.github.io/117-creating-a-sound-synthesizer-in-the-notebook/
from ipywidgets import Button, Box, Output
duration, sample_rate = .25, 44000.
t = np.linspace(0., duration, int(sample_rate*duration))
synth = lambda f: np.sin(2. * np.pi * f * t)
notes = 'C,C#,D,D#,E,F,F#,G,G#,A,A#,B,C'.split(',')
freqs = 440. * 2**(np.arange(3, 3 + len(notes)) / 12.)
buttons, layout_synth = [], Layout(width='30px', height='60px', border='1px solid black')
for note, freq in zip(notes, freqs):
    button = Button(description=note, layout=layout_synth)
    def on_button_clicked(f, b):
        with Output(): # suppress the audio widget output 
            display(Audio(synth(f), rate=sample_rate, autoplay=True))
    button.on_click(partial(on_button_clicked, freq))
    buttons.append(button)
Box(children=buttons)